## InfoDesReg: Dynamic Ensemble Selection for Regression tasks 

#### Example 

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore') 

In [2]:
import pandas as pd 

# models 
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression 

# metrics 
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo  


# local 
from infodesreg import DES

pd.set_option('display.max_columns', None)   

#### Load dataset 

In [3]:
from ucimlrepo import fetch_ucirepo  

abalone = fetch_ucirepo(id=1) 
X = abalone.data.features 
y = abalone.data.targets  

le = LabelEncoder() 
X['Sex'] = le.fit_transform(X['Sex'])  

In [4]:
TEST_SIZE = 0.2 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42) 
X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train, test_size=0.2, random_state=42)   

#### Define pool 

In [5]:
pool_models = [
    XGBRegressor(random_state=42), 
    RandomForestRegressor(random_state=42), 
    LinearRegression(), 
    KNeighborsRegressor(), 
]

for model in pool_models: 
    model.fit(X_train, y_train) 

### Define Dynamic Regressor model 

In [6]:
des = DES(pool_regressors=pool_models, 
          k=6, 
          knn_metric='minkowski', 
          metrics='mape', 
          threshold=0.1)

des.fit(X_dsel, y_dsel)

In [7]:
des.score(X_test, y_test)

4.874403324916552

#### XAI

In [8]:
index = 47

X_test.iloc[index]

Sex               2.0000
Length            0.5800
Diameter          0.4550
Height            0.1350
Whole_weight      0.7955
Shucked_weight    0.4050
Viscera_weight    0.1670
Shell_weight      0.2040
Name: 1338, dtype: float64

In [9]:
y_test.iloc[index]

Rings    10
Name: 1338, dtype: int64

In [10]:
prediction, contribution_df, neighbors_df= des.predict_xai(X_test.iloc[[index]])

prediction

     Rings
77       9
325     10
413      8
430      9
170      8
646     11


8.994432952472534

In [11]:
contribution_df

,selected_models,predictions,competences,weights
0,XGBRegressor,9.133,0.064,15.609
1,RandomForestRegressor,8.890,0.078,12.853
2,LinearRegression,8.913,0.090,11.141
3,KNeighborsRegressor,9.000,0.072,13.846


In [12]:
neighbors_df

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,target
77,2,0.565,0.440,0.125,0.8020,0.3595,0.1825,0.2150,9
325,2,0.550,0.425,0.150,0.8315,0.4110,0.1765,0.2165,10
413,2,0.560,0.415,0.130,0.7615,0.3695,0.1700,0.1955,8
430,2,0.545,0.410,0.120,0.7930,0.4340,0.1405,0.1900,9
170,2,0.560,0.415,0.145,0.8520,0.4300,0.1885,0.2050,8
646,2,0.540,0.420,0.135,0.8075,0.3485,0.1795,0.2350,11
